## Lisence

In [1]:
# Copyright 2017 The TensorFlow Authors. All Rights Reserved.
#
# Modifications copyright 2020 Dennis Adelved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

## Imports

In [482]:
import glob
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
import pandas as pd

import xml.etree.ElementTree as ET
import copy
import shutil

import re

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

#if StrictVersion(tf.__version__) < StrictVersion('1.12.0'):
  #raise ImportError('Please upgrade your TensorFlow installation to v1.12.*.')


# This is needed to display the images.
%matplotlib inline
# Load the TensorBoard notebook extension.
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [483]:
from utils import label_map_util
from utils import visualization_utils as vis_util

## Path Configuration

In [484]:
#Set the path to the folder containing the exported inference graph
MODEL_NAME = 'inference-graph-demo'

#adding the frozen inference graph to the path
PATH_TO_FROZEN_GRAPH = os.path.join(MODEL_NAME,'frozen_inference_graph.pb')

#Path to the label map
PATH_TO_LABELS = 'labelmap_plugs.pbtxt'

#Path to the images that are used for inference.
PATH_TO_TEST_IMAGES_DIR = 'images'


## Load a (frozen) Tensorflow model into memory.

In [485]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

In [486]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [487]:
def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

# Detection

In [488]:
#Run inference on image using the frozen inference graph
def run_inference_for_single_image(image, graph):


    if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[1], image.shape[2])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)


    image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')
    # Run inference
    output_dict = sess.run(tensor_dict,
                         feed_dict={image_tensor: image})

    # all outputs are float32 numpy arrays, so convert types as appropriate
    output_dict['num_detections'] = int(output_dict['num_detections'][0])
    output_dict['detection_classes'] = output_dict[
      'detection_classes'][0].astype(np.int64)
    output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
    output_dict['detection_scores'] = output_dict['detection_scores'][0]
    if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
    return output_dict

In [489]:
#Generate list of image paths
TEST_IMAGE_PATHS = []
for im in os.listdir(PATH_TO_TEST_IMAGES_DIR):
    if '.jpg' in im:
        TEST_IMAGE_PATHS.append(os.path.join(PATH_TO_TEST_IMAGES_DIR + '/' +  im))
        
#SET IMAGE SIZE
IMAGE_SIZE = (20,20)

#SET TRUE IF THE PREDICTED BOUNDING BOXES SHALL BE VISUALIZED ON THE INPUT IMAGE
VISUALIZE = False
        
#empty list for saving the output dicts for each prediction
dicts = []

image_dims = []


#Retrieve the detection data from the inference
with detection_graph.as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
        ops = tf.get_default_graph().get_operations()
        for op in ops: 
            op._set_device('/device:CPU:*')
        all_tensor_names = {output.name for op in ops for output in op.outputs}
        tensor_dict = {}
        for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
            tensor_name = key + ':0'
            if tensor_name in all_tensor_names:
                tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)

        for image_path in TEST_IMAGE_PATHS:

            #Convert RGB image to grayscale
            img = np.array(Image.open(image_path).convert('L'))
            
            #Expand the number of channels to 3, i.e. expand the grayscale channel.
            new_im = np.ndarray((img.shape[0],img.shape[1],3))
            for d in range(new_im.shape[-1]):
                new_im[:,:,d] = img[:,:]
            
            image_dims.append((new_im.shape[0],new_im.shape[1]))
            
            #Predict bounding boxes on grayscale image. 
            output_dict = run_inference_for_single_image(new_im[None,:,:,:], detection_graph)  
            
            
            #append detection from image to the detection list
            dicts.append(output_dict)
            
            
            
            #Visualize detected bounding boxes on RGB image
            visualize_image = np.array(Image.open(image_path))
            
            if VISUALIZE == True:
                vis_util.visualize_boxes_and_labels_on_image_array(
                  visualize_image,
                  output_dict['detection_boxes'],
                  output_dict['detection_classes'],
                  output_dict['detection_scores'],
                  category_index,
                  instance_masks=output_dict.get('detection_masks'),
                  use_normalized_coordinates=True,
                  line_thickness=4)
                plt.figure(figsize=IMAGE_SIZE)
                plt.imshow(visualize_image)

## Post-Processing Functions

In [528]:

def sorted_boxes(denrom,classes,sortby=0):
    concat = np.hstack((denorm,classes)).astype(int)
    sort_val = np.zeros_like(concat)
    sortind = np.argsort(denorm[:,0].argsort())
    for i in range(sortind.shape[0]):
        row = np.argwhere(sortind == i)
        sort_val[i,:] = concat[row,:]
    return sort_val

def pixel_to_depth(top,base,pixel_height):
    return (base-top) / pixel_height

def plug_to_depth(impath,boxes):
    im = np.asarray(Image.open(impath))
    
    top_str,base_str =impath.split('.')[0].split('_')[-2], (impath.split('.')[0].split('_')[-1])
    
    if ',' in top_str:
        top_str = top_str.split(',')[0] + '.' + top_str.split(',')[-1]

    if ',' in base_str:
        base_str = base_str.split(',')[0] + '.' + base_str.split(',')[-1]

    
    top,base = float(top_str), float(base_str)
    meter_per_pixel = pixel_to_depth(top,base,im.shape[0])
    mid_diff = ((boxes[:,2] - boxes[:,0]) / 2)
    mid = mid_diff + boxes[:,0]
    names = []
    depth = []
    pix = []
    image = []
    for i in range(len(mid)):

        names.append(class_to_name(boxes[i,-1]))
        depth.append(mid[i] * meter_per_pixel + top)
        pix.append(mid[i])
        image.append(impath)
        

    return names,depth,pix,image


#Denormalize the bounding box coordinates generated by the model
def denormalize(array,image_dims):
    if array.ndim==1:
        array=array[None,:]
    denorm = np.zeros_like(array)
    h,w = image_dims

    

    denorm[:,0] = array[:,0] * h
    denorm[:,1] = array[:,1] * w
    denorm[:,2] = array[:,2] * h
    denorm[:,3] = array[:,3] * w
    return denorm

    
#returns the filename and file extension from the path
def get_filename(path):
    filename = re.findall('^(.+)/([^/]+)$',path)
    filename = filename[0][1].split('.')
    return filenametop


def name_from_path(image_path):
    name_components = image_path.split('/')[-1].split('_')[0:3]
    name = name_components[0] + '_' + name_components[1] + '-' + name_components[2]
    top = (image_path.split('_')[-2])
    base = (image_path.split('_')[-1].split('.jpg')[0])
    return name,float(top),float(base)

def get_well(image_path,df):
    name,top,base = name_from_path(image_path)
    current_well = df[df['Well Name'] == name]
    g_top =  current_well[current_well['Measured Depth'] >= top]
    data = g_top[g_top['Measured Depth'] <= base]
    
    return data



def class_to_name(c):
    if c == 1:
        name = 'hplug'
        return name
    if c == 2:
        name = 'vplug'
        return name
    if c == 3: 
        name = 'scal'
        return name
    else:
        return 'Non-defined class'
    
    

def depth_conversion(df,plugtype):
    df = df[df['PlugType'] == plugtype].values
    pixel_diff = np.diff(df[:,2])
    depth_diff = np.diff(df[:,1])
    return depth_diff/pixel_diff





## Create Pixel-Depth mapping

In [539]:

lnames=[]
ldepth=[]
lpix=[]
limage=[]
for ind,d in enumerate(dicts):
    scores=np.argwhere(d['detection_scores'] > 0.5) 
    boxes = np.squeeze(d['detection_boxes'][scores]) 
    classes = d['detection_classes'][scores]
    denorm = denormalize(boxes,image_dims[ind]) 
    denorm = sorted_boxes(denorm,classes) 
    names,depth,pix,image = plug_to_depth(TEST_IMAGE_PATHS[ind],denorm)
    lnames += names
    ldepth += depth
    lpix += pix
    limage += image


d = {'Plug Type': lnames, 'Depth': ldepth, 'Pixel Location': lpix, 'Source': limage}
dataframe = pd.DataFrame(d)
dataframe = dataframe.sort_values('Depth').reset_index(drop=True)
dataframe.to_csv('pixel_depth_map.csv')
dataframe

,Plug Type,Depth,Pixel Location,Source
0,vplug,3632.026906,66.0,images/6407_1_3_3632_3633.jpg
1,hplug,3632.052385,128.5,images/6407_1_3_3632_3633.jpg
2,vplug,3632.324093,795.0,images/6407_1_3_3632_3633.jpg
3,hplug,3632.328577,806.0,images/6407_1_3_3632_3633.jpg
4,hplug,3632.757440,1858.0,images/6407_1_3_3632_3633.jpg
5,vplug,3632.798410,1958.5,images/6407_1_3_3632_3633.jpg
6,vplug,3632.917652,2251.0,images/6407_1_3_3632_3633.jpg
7,hplug,3632.974725,2391.0,images/6407_1_3_3632_3633.jpg
8,vplug,3634.041971,103.5,images/6407_1_3_3634_3635.jpg
9,scal,3634.061841,152.5,images/6407_1_3_3634_3635.jpg
